<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.4-Momentum.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Minbatch-Gradient und Momentum

- [Dive into Deep Learning, Zhang, Aston and Lipton, Zachary C. and Li, Mu and Smola, Alexander J.](https://d2l.ai/index.html)
- [Dive into Deep Learning](https://arxiv.org/abs/2106.11342)

Bisher sind wir beim gradientenbasierten Lernen auf zwei Extreme gestoßen: 
- das *Gradientenabstiegsverfahren* (GD, gradient descent) verwendet den gesamten Datensatz zur Berechnung von Gradienten und zur Aktualisierung von Parametern, und zwar einen Durchgang nach dem anderen. 
- Umgekehrt verarbeitet das *stochastische Gradientenabstiegsverfahren* SGD (stochastic gradient descent) eine Beobachtung nach der anderen, um Fortschritte zu erzielen. 

Jedes dieser Verfahren hat seine eigenen Nachteile. Der Gradientenabstieg ist nicht besonders *datensparend*, wenn die Daten sehr ähnlich sind. Der stochastische Gradientenabstieg ist nicht besonders *rechnereffizient*, da CPUs und GPUs nicht die volle Leistung der Vektorisierung ausschöpfen können. Dies deutet darauf hin, dass es einen goldenen Mittelweg geben könnte, und in der Tat ist es das, was wir bisher in den besprochenen Beispielen verwendet haben.

# Minibatch-Gradient

Die Verlustfunktion $\mathcal{L}$ beim Deep Learning hängt sowohl von den Parametern $\mathbf{w}$ als auch von den Daten $\mathbf{X}=\{ \mathbf{x},\mathbf{y} \}$ ab.

$$\mathcal{L}=f_{\mathbf{w}}(\mathbf{X})=f(\mathbf{X},\mathbf{w})$$

Im Falle einer Regression ist die Verlustfunktion beispielsweise der quadratische Fehler:

$$\mathcal{L}=\Vert \mathbf{x} \mathbf{w}-\mathbf{y} \Vert^2 $$

In der Vergangenheit war es selbstverständlich, dass wir zur Aktualisierung von Parametern eher *Minibatches* von Daten als einzelne Beobachtungen lesen. Bei der Verarbeitung einzelner Beobachtungen müssen wir viele einzelne Matrix-Vektor- (oder sogar Vektor-Vektor-) Multiplikationen durchführen, was ziemlich teuer ist und einen erheblichen Overhead für das zugrunde liegende Deep-Learning-System bedeutet. Dies gilt sowohl für die Bewertung eines Netzes, wenn es auf Daten angewendet wird (oft als Inferenz bezeichnet), als auch für die Berechnung von Gradienten zur Aktualisierung von Parametern $\mathbf{w}$. 





Das heisst, dies gilt immer dann, wenn wir folgenden Update durchführen:
$$\mathbf{w} \leftarrow \mathbf{w} - \eta_t \mathbf{g}_t,$$
wobei
$$\mathbf{g}_t = \partial_{\mathbf{w}} f(\mathbf{x}_{t}, \mathbf{w})$$

Wir können die *Rechnereffizienz* dieser Operation erhöhen, indem wir sie jeweils auf ein **Minibatch von Beobachtungen** $\mathcal{B}$ anwenden. Das heisst, wir ersetzen den Gradienten $\mathbf{g}_t$ über eine einzelne Beobachtung durch einen über eine kleine Gruppe

$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

Sehen wir uns an, was dies mit den statistischen Eigenschaften von $\mathbf{g}_t$ macht: Da sowohl $\mathbf{x}_t$ als auch alle Elemente der Minibatch $\mathcal{B}_t$ gleichmässig zufällig aus der Trainingsmenge gezogen werden, bleibt der Erwartungswert des Gradienten unverändert. Die Varianz hingegen wird deutlich reduziert. Da der Minibatch-Gradient aus $b := |\mathcal{B}_t|$ unabhängigen Gradienten besteht, die gemittelt werden, wird seine Standardabweichung um den Faktor $\frac{1}{\sqrt{b}}$ verringert. Dies ist an sich eine gute Sache, da es bedeutet, dass die Aktualisierungen zuverlässiger auf den vollständigen Gradienten abgestimmt sind.



- Naiv betrachtet würde dies bedeuten, dass die Wahl einer grossen Minibatch $\mathcal{B}_t$ allgemein wünschenswert wäre. Leider ist ab einem gewissen Punkt die zusätzliche Verringerung der Standardabweichung minimal im Vergleich zum linearen Anstieg der Rechenkosten. 
- In der Praxis wählen wir einen Ministapel, der gross genug ist, um eine gute Berechnungseffizienz zu bieten und dennoch in den Speicher einer GPU zu passen. 

# Momentum

In Abschnitt über das stochastische Gradientenabstiegsverfahren haben wir untersucht, was passiert, wenn man einen stochastischen Gradientenabstieg durchführt, d.h. wenn man eine Optimierung durchführt, bei der nur eine verrauschte Variante des Gradienten verfügbar ist. Insbesondere haben wir festgestellt, dass wir bei verrauschten Gradienten besonders vorsichtig sein müssen, wenn es darum geht, die Lernrate angesichts des Rauschens zu wählen. Wenn wir sie zu schnell verringern, gerät die Konvergenz ins Stocken. Wenn wir zu nachsichtig sind, konvergieren wir nicht zu einer ausreichend guten Lösung, da das Rauschen uns immer weiter von der Optimalität wegführt.


## Leaky Averages

Das Minibatch-SGD-Verfahren ist ein Mittel zur Beschleunigung zur der Optimierung. Dies hatte auch den angenehmen Nebeneffekt, dass die Mittelwertbildung von Gradienten die Varianz reduzierte. Der stochastische Minibatch-Gradientenabstieg kann wie folgt berechnet werden:

$$\mathbf{g}_{t, t-1} = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w}_{t-1}) = \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} \mathbf{h}_{i, t-1}.
$$

Um die Notation einfach zu halten, verwenden wir hier $\mathbf{h}_{i, t-1} = \partial_{\mathbf{w}} f(\mathbf{x}_i, \mathbf{w}_{t-1})$ als stochastischen Gradientenabstieg für die Stichprobe $i$ unter Verwendung der zum Zeitpunkt $t-1$ aktualisierten Gewichte.


Es wäre schön, wenn wir den Effekt der Varianzreduktion auch über die Mittelung von Gradienten auf einer Minibatch hinaus nutzen könnten. Eine Möglichkeit, diese Aufgabe zu erfüllen, besteht darin, die Gradientenberechnung durch einen **"Leaky Average"** zu ersetzen:

$$\mathbf{v}_t = \beta \mathbf{v}_{t-1} + \mathbf{g}_{t, t-1}$$

für irgendein $\beta \in (0, 1)$. Damit wird der momentane Gradient durch einen Gradienten ersetzt, der über mehrere *vergangene* Gradienten gemittelt wurde. $\mathbf{v}$ wird *Momentum* genannt. Er akkumuliert vergangene Gradienten, ähnlich wie ein schwerer Ball, der die Zielfunktionslandschaft hinunterrollt, über vergangene Kräfte integriert. Um zu sehen, was hier im Detail passiert, erweitern wir $\mathbf{v}_t$ rekursiv zu

$$\begin{aligned}
\mathbf{v}_t = \beta^2 \mathbf{v}_{t-2} + \beta \mathbf{g}_{t-1, t-2} + \mathbf{g}_{t, t-1}
= \ldots, = \sum_{\tau = 0}^{t-1} \beta^{\tau} \mathbf{g}_{t-\tau, t-\tau-1}.
\end{aligned}$$



- Ein grosses $\beta$ läuft auf einen langreichweitigen Mittelwert hinaus, während kleines $\beta$ nur eine geringe Korrektur gegenüber einer Gradientenmethode bedeutet. 
- Der neue Gradientenersatz zeigt nicht mehr in die Richtung des steilsten Abstiegs auf einer bestimmten Instanz, sondern in die Richtung eines gewichteten Mittelwerts der vergangenen Gradienten. 
- Auf diese Weise können wir die meisten Vorteile der Mittelwertbildung über einen Stapel nutzen, ohne die Kosten für die eigentliche Berechnung der Gradienten auf diesem Stapel. 

Die obigen Überlegungen bildeten die Grundlage für das, was heute als *beschleunigte* Gradientenmethoden bekannt ist, wie z. B. die Gradienten mit Momentum. 
1.  Sie haben den zusätzlichen Vorteil, dass sie in Fällen, in denen das Optimierungsproblem schlecht konditioniert ist (d. h. in denen es einige Richtungen gibt, in denen der Fortschritt viel langsamer ist als in anderen, ähnlich einer engen Schlucht), viel effektiver sind. 
2. Darüber hinaus ermöglichen sie eine Mittelwertbildung über nachfolgende Gradienten, um stabilere Abstiegsrichtungen zu erhalten. In der Tat ist der Aspekt der Beschleunigung selbst bei rauschfreien konvexen Problemen einer der Hauptgründe, warum Momentum funktioniert und warum es so gut funktioniert.

### Weiterführende Literatur

Wie nicht anders zu erwarten, ist Momentum aufgrund seiner Wirksamkeit ein gut erforschtes Thema in der Optimierung für Deep Learning und darüber hinaus. Siehe z.B. den schönen Aufsatz [Why Momentum Really Works](https://distill.pub/2017/momentum/) von Gabriel Goh für eine eingehende Analyse und interaktive Animation. 

Sie wurde von Boris T. Polyak [1] 1964 vorgeschlagen.
Yurii Nesterov [2] hat eine detaillierte theoretische Diskussion im Kontext der konvexen Optimierung durchgeführt. 
Es ist seit langem bekannt, dass Momentum beim Deep Learning von Vorteil ist. Siehe z.B. die Diskussion von Sutskever, Martens und Dahl [3] für Details.

[1] [Polyak, Boris T.: Some methods of speeding up the convergence of iteration, USSR Computational Mathematics and Mathematical Physics)](https://www.sciencedirect.com/science/article/abs/pii/0041555364901375)

[2] [Nesterov, Yurii: Lectures on convex optimization, Spinger (2018)](https://link.springer.com/book/10.1007/978-3-319-91578-4?gclid=Cj0KCQiAoNWOBhCwARIsAAiHnEhrNoalLKkTumHQLH_V0aIj3K5pyBpDvEAoFicgCFbV_2N8ZtYdFigaAs4ZEALw_wcB)

[3] [Sutskever, Ilya and Martens, James and Dahl, George and Hinton, Geoffrey: On the importance of initialization and momentum in deep learning](https://proceedings.mlr.press/v28/sutskever13.html)


### Ein schlecht konditioniertes Problem

Um ein besseres Verständnis für die geometrischen Eigenschaften der Momentum-Methode zu bekommen, greifen wir den Gradientenabstieg wieder auf, allerdings mit einer deutlich weniger angenehmen Zielfunktion. Erinnern Sie sich, dass wir für das SGD-Verfahren $f(\mathbf{x}) = x_1^2 + 2 x_2^2$ verwendet haben, d.h. eine mässig verzerrte ellipsoidische Zielfunktion. 

Wir verzerren diese Funktion weiter, indem wir sie in Richtung $x_1$ strecken über

$$f(\mathbf{x}) = 0.1 x_1^2 + 2 x_2^2.$$

Wie zuvor hat $f$ sein Minimum bei $(0, 0)$. Diese Funktion ist *sehr* flach in Richtung von $x_1$. Sehen wir uns an, was passiert, wenn wir den Gradientenabstieg wie zuvor auf dieser neuen Funktion durchführen. Wir wählen eine Lernrate von $0.4$.

In [ ]:
# !pip install requests
# !pip install autograd
# !git clone https://github.com/dsgiitr/d2l-pytorch.git
# Homepage
# https://d2l.ai/

# %pip install d2l # geht nur mit 3.11.9 python version

In [ ]:
%matplotlib inline
import torch
import d2l
import numpy as np
import matplotlib.pyplot as plt 


eta = 0.4


def f_2d(x1, x2):
    return 0.1 * x1**2 + 2 * x2**2


def gd_2d(x1, x2, s1, s2):
    return (x1 - eta * 0.2 * x1, x2 - eta * 4 * x2, 0, 0)

plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))


Konstruktionsbedingt ist der Gradient in der $x_2$-Richtung *viel* höher und ändert sich viel schneller als in der horizontalen $x_1$-Richtung. Wir sitzen also zwischen zwei unerwünschten Möglichkeiten fest: Wenn wir eine kleine Lernrate wählen, stellen wir sicher, dass die Lösung nicht in $x_2$-Richtung divergiert, aber wir müssen mit einer langsamen Konvergenz in $x_1$-Richtung rechnen. Umgekehrt kommen wir bei einer grossen Lernrate in der Richtung $x_1$ schnell voran, divergieren aber in $x_2$. 

Das folgende Beispiel zeigt, was selbst nach einer leichten Erhöhung der Lernrate von $0.4$ auf $0.6$ passiert. Die Konvergenz in $x_1$-Richtung verbessert sich, aber die Qualität der Gesamtlösung ist viel schlechter.


In [ ]:
eta = 0.6
plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))


### The Momenten-Methode

Mit der Momenten-Methode können wir das oben beschriebene Problem des Gradientenabstiegs lösen.

Betrachtet man die obige Optimierungskurve, so könnte man intuitiv annehmen, dass eine Mittelung der Gradienten über die Vergangenheit gut funktionieren würde. Schliesslich werden dadurch in der Richtung $x_1$ gut ausgerichtete Gradienten aggregiert, wodurch sich die zurückgelegte Strecke mit jedem Schritt vergrössert. In $x_2$-Richtung, wo die Gradienten oszillieren, wird ein aggregierter Gradient die Schrittweite aufgrund der sich gegenseitig aufhebenden Oszillationen verringern.
Die Verwendung von $\mathbf{v}_t$ anstelle des Gradienten $\mathbf{g}_t$ führt zu den folgenden Aktualisierungsgleichungen:

$$
\begin{aligned}
\mathbf{v}_t &\leftarrow \beta \mathbf{v}_{t-1} + \mathbf{g}_{t, t-1}, \\\
\mathbf{x}_t &\leftarrow \mathbf{x}_{t-1} - \eta_t \mathbf{v}_t.
\end{aligned}
$$

Beachten Sie, dass wir für $\beta = 0$ einen regulären Gradientenabstieg erhalten. Bevor wir uns näher mit den mathematischen Eigenschaften befassen, wollen wir einen kurzen Blick darauf werfen, wie sich der Algorithmus in der Praxis verhält.


In [ ]:
def momentum_2d(x1, x2, v1, v2):
    v1 = gamma * v1 + eta * 0.2 * x1
    v2 = gamma * v2 + eta * 4 * x2
    return x1 - v1, x2 - v2, v1, v2


eta, gamma = 0.4, 0.5

plt.figure(figsize=(10, 6))
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))


Wie wir sehen können, konvergiert der Impuls auch bei der gleichen Lernrate wie zuvor gut. Schauen wir uns an, was passiert, wenn wir den Impulsparameter verringern. Eine Halbierung auf $\beta = 0.25$ führt zu einer Trajektorie, die kaum konvergiert. Nichtsdestotrotz ist sie viel besser als ohne Momentum (wenn die Lösung divergiert).


In [ ]:
eta = 0.6
plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))


Man beachte, dass man Momentum mit stochastischem Gradientenabstieg und insbesondere mit stochastischem Minibatch-Gradientenabstieg kombinieren kann. Die einzige Änderung ist, dass wir in diesem Fall die Gradienten $\mathbf{g}_{t, t-1}$ durch $\mathbf{g}_t$ ersetzen. Schließlich initialisieren wir der Einfachheit halber $\mathbf{v}_0 = 0$ zum Zeitpunkt $t=0$. Schauen wir uns an, was Leaky Averaging tatsächlich mit den Aktualisierungen macht.

### Effektives Stichprobengewicht

Wir erinnern uns, dass $\mathbf{v}_t = \sum_{\tau = 0}^{t-1} \beta^{\tau} \mathbf{g}_{t-\tau, t-\tau-1}$. Im Grenzfall summieren sich die Terme zu $\sum_{\tau=0}^\infty \beta^\tau = \frac{1}{1-\beta}$. Mit anderen Worten, anstatt einen Schritt der Größe $\eta$ beim Gradientenabstieg oder stochastischen Gradientenabstieg zu machen, machen wir einen Schritt der Größe $\frac{\eta}{1-\beta}$, während wir es gleichzeitig mit einer potentiell viel besser verhaltenden Abstiegsrichtung zu tun haben. Dies sind zwei Vorteile in einem. Um zu veranschaulichen, wie sich die Gewichtung bei verschiedenen Entscheidungen für $\beta$ verhält, betrachten Sie das folgende Diagramm.


In [ ]:
gammas = [0.95, 0.9, 0.6, 0]
d2l.set_figsize((3.5, 2.5))
for gamma in gammas:
    x = np.arange(40)
    d2l.plt.plot(x, gamma**x, label="gamma = %.2f" % gamma)

d2l.plt.xlabel("time")
d2l.plt.legend()




## Zusammenfassung

* Momentum ersetzt die Gradienten durch einen **leaky average** über die vergangenen Gradienten. Dies beschleunigt die Konvergenz erheblich.
* Es ist sowohl für den rauschfreien Gradientenabstieg als auch für den (verrauschten) stochastischen Gradientenabstieg wünschenswert.
* Momentum verhindert, dass der Optimierungsprozess ins Stocken gerät, was beim stochastischen Gradientenabstieg viel wahrscheinlicher der Fall ist.
* Die effektive Anzahl der Gradienten ist durch $\frac{1}{1-\beta}$ aufgrund der exponentiellen Abwärtsgewichtung der Vergangenheitsdaten gegeben.
* Im Fall von konvexen quadratischen Problemen kann dies explizit im Detail analysiert werden.
* Die Implementierung ist recht einfach, erfordert aber die Speicherung eines zusätzlichen Zustandsvektors (Impuls $\mathbf{v}$).

